In [1]:
import pandas as pd
import numpy as np
import PyPDF2 as pdf
import os
from tqdm import tqdm
import re
import json
import collections


# pdfminer.six
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
import pdfplumber

## 功能
- 將分好類的至股東報告書擷取段落

## 要改的
- 加一個type選擇棄，kw2改成list-->ok
- cut_paragraph 在進入kw2十自動加上k1的起始位置-->ok
- 寫一個標題分類氣-->ok
- 條件二符合的去除實施成果-->ok
- 同一條件同位置多筆資料屬同一kw1去除重複-->ok
- 小於100字標記-->ok
- para0計算-->ok


### 準備檔案

In [2]:
df = pd.read_excel('頁碼簿.xlsx', dtype='str')
df[['startpage', 'endpage']] = df[['startpage', 'endpage']].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1712 entries, 0 to 1711
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   co_id      1712 non-null   object
 1   year       1712 non-null   object
 2   ext        1712 non-null   object
 3   filename   1712 non-null   object
 4   isedible   1712 non-null   object
 5   startpage  1712 non-null   int32 
 6   endpage    1712 non-null   int32 
dtypes: int32(2), object(5)
memory usage: 80.4+ KB


In [3]:
inputroot = os.getcwd()
df['inputpath'] = ''
df['filename_folder'] = ''

In [4]:
count=0
colist = []
# 讀取文件夾
for a,b,c in os.walk(inputroot):
    count+=1
    if count<=1:
        continue
    d = os.path.basename(a)
    for file in c:
        try:
            row = df.index[df['filename'].str.slice(stop=20)==file[:20]].tolist()[0]
            s = df.loc[row,'startpage']
            endpage =  df.loc[row,'endpage']
            e = str(int(endpage)-1)
            fn,ext = file.split('.')
            df.loc[row,'inputpath'] = a+'/'+fn+'.'+ext
            df.loc[row,'filename_folder'] = file
        except Exception as e: # cannot find
#             print(e)
            continue
df.head()

,co_id,year,ext,filename,isedible,startpage,endpage,inputpath,filename_folder
0,1102.0,2019,pdf,2019_1102_20200623F04_20211212_154547.pdf,True,5,7,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102/20...,2019_1102_20200623F04_20211212_154547.pdf
1,1102.0,2020,pdf,2020_1102_20210625F04_20211212_154258.pdf,True,5,11,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102/20...,2020_1102_20210625F04_20211212_154258.pdf
2,1103.0,2019,pdf,2019_1103_20200622F04_20211212_155216.pdf,True,4,5,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103/20...,2019_1103_20200622F04_20211212_155216.pdf
3,1103.0,2020,pdf,2020_1103_20210617F04_20211212_155303.pdf,True,6,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103/20...,2020_1103_20210617F04_20211212_155303.pdf
4,1108.0,2019,pdf,2019_1108_20200618F04_20211212_155632.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1108/20...,2019_1108_20200618F04_20211212_155632.pdf


### 分段器

In [5]:
def extract_text_by_page(pdf_path,start,end, maxpages=0, pagenos=None): # extract whole document
    with pdfplumber.open(pdf_path) as fh:
        text_by_page = ''
        for i in range(start,end):
            text = fh.pages[i]
            text_by_page += text.extract_text()
            text_by_page = text_by_page.replace('\n','')
    fh.close()
    return text_by_page

In [6]:
def cut_paragraph(pdf_file,start,end):
    
    texts = extract_text_by_page(pdf_file,start,end)
    paragraph = dict()
    
    loc = {'1':-1, '2':-1, '3':-1, '4':-1}
    text=seq=exception=score=fseq=''
  
    for t in texts:
        text+=t
#     print(text)
    
    # 列舉全部情況
    paragraph['sit1'] = sit1loc(text)
    paragraph['sit2'] = sit2loc(text)
    paragraph['sit3'] = sit3loc(text)
    paragraph['sit4'] = sit4loc(text)
    
    # classify letter type
    classification_type, score = letter_classification(paragraph)
    
    #debug
#     print(score)
#     print(classification_type)
#     print(paragraph)
#     print(text)
#     return loc, paragraph, score, seq, fseq, text, exception

    # 預測段落順序
    if score==4: # 如果有任何一條件是完全找不到關鍵字
        seq = sequence_speculate(dump_paragraph(paragraph))
        loc, exception = cut_paragraph_by_sequence(dump_paragraph(paragraph),sequence=seq)
        fseq=''
        for item in sorted(loc.items(),key=lambda t:t[1], reverse=False):
            fseq+=item[0]
            
        # if paragraph too short, exception
        if location_dist_too_short(loc,len(text), fseq):
            exception+='~~dist too short'
    else:
        exception = '~~some situation not found'
        
    
    return loc, paragraph, score, seq, fseq, text, exception


In [7]:
def sit1loc(line):
    kw1 = ['營業', '營運', '年', '經營','實施']
    kw2 = ['結果', '成果', '報告', '概況', '狀況', '情形', '績效', '收入']
    matches = list()
    interval = 5
    # 只找後面
    for a in kw1:   
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start():match.end()+interval]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start():match.end()+interval])])

    return matches # NOT FOUND

def sit2loc(line):
    kw1 = ['營業', '營運', '年', '經營']
    kw2 = ['概要', '計畫', '計劃', '概述', '方針']
    matches = list()
    interval = 10
    # 只找後面
    for a in kw1:       
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start():match.end()+interval]) is not None:# found 2nd kw
#                     print(match, line[match.start():match.end()+interval])
                    if sit2locexclude1(line[match.start():]):
                        continue
                    else:
                        matches.append(match)
                        matches.append([i for i in re.finditer(b, line[match.start():match.end()+interval])])
    return matches

def sit2locexclude1(line):
    kw1 = ['實施']
    kw2 = ['結果', '成果']
    matches = list()
    interval = 20
    # 只找後面
    for a in kw1:   
        kw_1st = re.finditer(a,line[:interval])
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[:interval]) is not None:# found 2nd kw
                    return 1

    return 0 # NOT FOUND

def sit3loc(line):
    kw1 = ['未來']
    kw2 = ['發展', '展望', '策略', '目標', '方向']
    matches = list()
    interval_b = 5
    interval_a = 15
    # 前後
    for a in kw1:
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start()-interval_b:match.end()+interval_a]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start()-interval_b:match.end()+interval_a])])
    return matches

def sit4loc(line):
    kw1 = ['外部']
    kw2 = ['環境', '影響']
    matches = list()
    interval = 20
    # 前後
    for a in kw1:
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start()-interval:match.end()+interval]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start()-interval:match.end()+interval])])
    return matches


In [8]:
def sequence_speculate(paragraph):# 推測段落的編寫順序
    # 將段落集排序，list內容drop掉    
    paragraph = drop_duplicated_keyword(paragraph)
    
    
    # 如果以中位數來區分
#     print(paragraph)
#     from statistics import median
#     lmedian1 = median(paragraph['sit1'])
#     lmedian2 = median(paragraph['sit2'])
#     lmedian3 = median(paragraph['sit3'])
#     lmedian4 = median(paragraph['sit4'])
    
#     d = {'1':lmedian1,'2':lmedian2,'3':lmedian3,'4':lmedian4}
#     print(d)
#     seq = ''
#     for item in sorted(d.items(),key=lambda t:t[1], reverse=False):
#         seq+=item[0]
#     return seq
    
    # 如果以大數法則來區分
    l1 = len(paragraph['sit1'])
    l2 = len(paragraph['sit2'])
    l3 = len(paragraph['sit3'])
    l4 = len(paragraph['sit4'])    
    lmax = max(l1,l2,l3,l4)

    combinations = list()
    for i in range(lmax):
        if i<l1:
            c1 = paragraph['sit1'][i]
        if i<l2:
            c2 = paragraph['sit2'][i]
        if i<l3:
            c3 = paragraph['sit3'][i]
        if i<l4:
            c4 = paragraph['sit4'][i]
        
        d = {'1':c1,'2':c2,'3':c3,'4':c4}
        combinations.append(collections.OrderedDict(sorted(d.items(),key=lambda t:t[1])))
    
    ctypes = {}
    for combin in combinations:
#         print(combin)
        r = ''
        for k,v in combin.items():
            r += k
        if r in ctypes:
            ctypes[r]+=1
        else:
            ctypes[r]=1
    return sorted(ctypes.items(),key=lambda t:t[1], reverse=True)[0][0]






def title_discrimination(c1,c2,c3,c4):
    cmax = max(c1,c2,c3,c4)
    cmin = min(c1,c2,c3,c4)
    cavg = (c1+c2+c3+c4)/4
    
    # 重新排序，不管情況重小排到大
    d = {'1':c1, '2':c2, '3':c3, '4':c4}
    d = sorted(d.items(),key=lambda t:t[1])
    seq=''
    for item in d:
        seq +=item[0]
#     print(seq)
    #只要含有-1就退出
    if any([c1==-1, c2==-1, c3==-1, c4==-1]):
        return -1,seq
    # check full dist -->too short return 0
    if cmax-cmin<100:
        return 1,seq
    else:
        # check ---1-------2-3-4---   
        if (d[3][1] - d[1][1])<50: 
            return 2,seq
        # check ---1-2-3-------4---  
        elif (d[2][1] - d[0][1])<50: 
            return 3,seq
        # check ---1-2-----3---4---  
        elif (d[1][1] - d[0][1])<50: 
            return 4,seq
        # check ---1---2-----3-4---
        elif (d[3][1] - d[2][1])<50: 
            return 5,seq
        # check ---1----2-3----4---
        elif (d[2][1] - d[1][1])<50: 
            return 6,seq
        # else  ---1---2---3---4---
        else: 
            return 0,seq

In [9]:
def cut_paragraph_by_sequence(paragraph, sequence='1234'):
    # 將段落集排序，list內容drop掉    
    paragraph = drop_duplicated_keyword(paragraph)

    # 定義location 無序        
    maxlocation = {
        '1':len(paragraph['sit1']),
        '2':len(paragraph['sit2']),
        '3':len(paragraph['sit3']),
        '4':len(paragraph['sit4'])        
    }
    currentlocation = {'1':0,'2':0,'3':0,'4':0}
    classlocation = {
        '1':paragraph['sit1'][currentlocation['1']],
        '2':paragraph['sit2'][currentlocation['2']],
        '3':paragraph['sit3'][currentlocation['3']],
        '4':paragraph['sit4'][currentlocation['4']]
    }
#     print(classlocation)
            
    # 有序
    isTitle, seqs = title_discrimination(classlocation[sequence[0]],classlocation[sequence[1]],classlocation[sequence[2]],classlocation[sequence[3]])
    status = True
    exception = ''
    while isTitle!=0 and isTitle!=-1 and status:
#         print('maxlocation', maxlocation)
#         print('currentlocation', currentlocation)
#         print(isTitle, seqs)
#         print('before_classlocation', classlocation)
        if all([currentlocation['1']>=maxlocation['1'],
               currentlocation['2']>=maxlocation['2'],
               currentlocation['3']>=maxlocation['3'],
               currentlocation['4']>=maxlocation['4'] ]):
            exception = 'max limit reached'
            break
        if isTitle==1: #全改
            for i in range(4): # 改後面三個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break
        elif isTitle==2:
            for i in range(4): # 改後面三個
                tempint = int(seqs[i])-1
                if i==0: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break

        elif isTitle==3:
            for i in range(3): # 改前面三個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break

        elif isTitle==4:
            for i in range(2): # 改前面兩個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break

        elif isTitle==5:
            for i in range(4): # 改後面兩個
                tempint = int(seqs[i])-1
                if i==0 or i==1: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break
        elif isTitle==6:
            for i in range(3): # 改中間兩個
                tempint = int(seqs[i])-1
                if i==0: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break
#         print('after_classlocation', classlocation)
        isTitle, seqs = title_discrimination(classlocation[sequence[0]],classlocation[sequence[1]],classlocation[sequence[2]],classlocation[sequence[3]])
#         print(isTitle, seqs,exception)
#         print('--------')

    # 檢查順序性
    # 1,2,3,4段具順序性
    # 如果兩個數其中一個-1就跳過
    while classlocation[sequence[1]]<classlocation[sequence[0]] and classlocation[sequence[1]]!=-1 and classlocation[sequence[0]] !=-1:
        currentlocation[sequence[1]] +=1
        if currentlocation[sequence[1]]<maxlocation[sequence[1]]:
            classlocation[sequence[1]] = paragraph['sit'+sequence[1]][currentlocation[sequence[1]]]
        else: break
    while classlocation[sequence[2]]<classlocation[sequence[1]] and classlocation[sequence[2]]!=-1 and classlocation[sequence[1]] !=-1:
        currentlocation[sequence[2]] +=1
        if currentlocation[sequence[2]]<maxlocation[sequence[2]]:
            classlocation[sequence[2]] = paragraph['sit'+sequence[2]][currentlocation[sequence[2]]]
        else: break
    while classlocation[sequence[3]]<classlocation[sequence[2]] and classlocation[sequence[3]]!=-1 and classlocation[sequence[2]] !=-1:
        currentlocation[sequence[3]] +=1
        if currentlocation[sequence[3]]<maxlocation[sequence[3]]:
            classlocation[sequence[3]] = paragraph['sit'+sequence[3]][currentlocation[sequence[3]]]
        else: break
    
    
    return classlocation, exception


In [10]:
def letter_classification(paragraph):
    classification = {'c1':0, 'c2':0, 'c3':0, 'c4':0}
    
    if len(paragraph['sit1'])!=0:
        classification['c1'] = 1
    if len(paragraph['sit2'])!=0:
        classification['c2'] = 1
    if len(paragraph['sit3'])!=0:
        classification['c3'] = 1
    if len(paragraph['sit4'])!=0:
        classification['c4'] = 1

    score=0
    for k,v in classification.items():
        score+=v
         
    return classification, score

def location_dist_too_short(loc, total, fseq):
    # if distance between two paragraph is too short return 1
    # else return 0
    count=0
    for index in fseq:
        if count==0:
            pre = loc[index]
            count+=1
            continue
        else:
            dist=loc[index]-pre
            if dist<100:
                return 1
            pre = loc[index]
            count+=1
    
    if total-pre<100:
        return 1
    
    return 0
        

In [11]:
def dump_paragraph(_obj):
    # 將match物件轉換為dict與list的組合
    
    if type(_obj) ==type(dict()):
        obj = _obj.copy()  # 為避免傳址問題, 但遇到dict以外物件會出現錯誤
        for k,v in obj.items():
            templist = list()
            for i in v:
                if type(i)==type(list()):
                    templist2 = list()
                    for i2 in i:
                        templist2.append({'key':i2.group(),'start':i2.start(),'end':i2.end()})
                    templist.append(templist2)
                else:
                    templist.append({'key':i.group(),'start':i.start(),'end':i.end()})
            obj[k] = templist
    else:
        obj = str(_obj)

    return obj

def drop_duplicated_keyword(paragraph):
    # 將段落集排序，list內容drop掉    
    for k,v in paragraph.items():
        templist = list()
        for items in v:
            if type(items)==type(list()):
                pass
            else:
                templist.append(items['start'])
        paragraph[k] = sorted(list(set(templist)))
        
#     print(paragraph)
    return paragraph

### main

In [12]:
df['text'] = ''

# 開始分類每個檔案
count=0
pbar = tqdm(total=len(df))

for label, row in df.iterrows():  
    count+=1
    l=p=s=f=t=e=''
    
    try:
        file = row['inputpath']
        l,p,sc,se,f,t,e = cut_paragraph(file,row['startpage']-1,row['endpage']-1)
        #t= cut_paragraph(file,row['startpage']-1,row['endpage'])
    except Exception as ex:
        l = (-1,-1,-1,-1)
        e = str(ex)

    
    df.loc[label, 'exception'] = str(e)
    df.loc[label,'location'] = json.dumps(l)
    df.loc[label,'text'] = t
    df.loc[label,'paragraph'] = json.dumps(dump_paragraph(p))
    df.loc[label, 'score'] = sc
    df.loc[label,'sequence_projected'] = se
    df.loc[label,'sequence_final'] = f

    pbar.update(1)
    
#     if count==30:
#         break
pbar.close()


100%|████████████████████████████████████████████████████████████████████████████| 1712/1712 [1:18:49<00:00,  2.76s/it]


In [13]:
df

,co_id,year,ext,filename,isedible,startpage,endpage,inputpath,filename_folder,text,exception,location,paragraph,score,sequence_projected,sequence_final
0,1102.0,2019,pdf,2019_1102_20200623F04_20211212_154547.pdf,True,5,7,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102/20...,2019_1102_20200623F04_20211212_154547.pdf,公司簡介 壹、致股東報告書 壹、2019 年國際暨國內經濟情勢回顧 ...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 167...",1.0,,
1,1102.0,2020,pdf,2020_1102_20210625F04_20211212_154258.pdf,True,5,11,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102/20...,2020_1102_20210625F04_20211212_154258.pdf,致股東報告書 致股東報告書 壹、2020 年國際暨國內經濟情勢回顧 ...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 155...",3.0,,
2,1103.0,2019,pdf,2019_1103_20200622F04_20211212_155216.pdf,True,4,5,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103/20...,2019_1103_20200622F04_20211212_155216.pdf,1 致股東報告書 致股東報告書 11. 致股東報告書各位股東女士、先生： (1) 主要產銷情...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 568...",2.0,,
3,1103.0,2020,pdf,2020_1103_20210617F04_20211212_155303.pdf,True,6,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103/20...,2020_1103_20210617F04_20211212_155303.pdf,1.致股東報告書各位股東女士、先生： 民國(下同)109 年新冠病毒在全球大流行，各國...,~~dist too short,"{""1"": 1001, ""2"": 2656, ""3"": 2234, ""4"": 2162}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 100...",4.0,1432,1432
4,1108.0,2019,pdf,2019_1108_20200618F04_20211212_155632.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1108/20...,2019_1108_20200618F04_20211212_155632.pdf,壹、致股東報告書 一、董事長致詞 各位股東女士、先生：大家好！ 本公司一○八年全年度合併營業...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 44,...",2.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,9942.0,2020,pdf,2020_9942_20210610F04_20211220_220634.pdf,True,5,8,C:\Users\DC\辰瑜的論文\年報108-109年\M9900其他\9942-zip/...,2020_9942_20210610F04_20211220_220634.pdf,壹、致股東報告書一、一百零九年度營業報告：1、營業計劃實施成果本公司109年度合併營業收入為...,,"{""1"": 14, ""2"": 405, ""3"": 1341, ""4"": 1522}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 16,...",4.0,1234,1234
1708,9944.0,2019,pdf,2019_9944_20200619F04_20211220_220633.pdf,True,5,10,C:\Users\DC\辰瑜的論文\年報108-109年\M9900其他\9944/2019...,2019_9944_20200619F04_20211220_220633.pdf,壹、致股東報告書 ...,,"{""1"": 69, ""2"": 1604, ""3"": 3188, ""4"": 4092}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 71,...",4.0,1234,1234
1709,9945.0,2019,pdf,2019_9945_20200609F04_20211220_220921.pdf,True,4,6,C:\Users\DC\辰瑜的論文\年報108-109年\M9900其他\9945/2019...,2019_9945_20200609F04_20211220_220921.pdf,壹、致股東報告書 敬愛的各位股東女士、先生們〆 一、民國 108 年度營業報告 (一...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 35,...",3.0,,
1710,9945.0,2020,pdf,2020_9945_20210615F04_20211220_221142.pdf,True,4,6,C:\Users\DC\辰瑜的論文\年報108-109年\M9900其他\9945/2020...,2020_9945_20210615F04_20211220_221142.pdf,壹、致股東報告書 敬愛的各位股東女士、先生們〆 一、民國 109 年度營業報告 (一...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 35,...",3.0,,


In [14]:
df['exception'].value_counts()

                                         703
~~some situation not found               681
~~dist too short                         173
~~list out of range 5~~dist too short     65
~~list out of range 5                     44
~~list out of range 6                     12
~~list out of range 6~~dist too short     11
~~list out of range 2~~dist too short      7
~~list out of range 3~~dist too short      6
~~list out of range 4~~dist too short      4
~~list out of range 1~~dist too short      3
~~list out of range 4                      2
~~list out of range 2                      1
Name: exception, dtype: int64

#### 後處理

In [15]:
df['Classified'] = None
for label, row in df.iterrows():
    if df.loc[label,'score'] == 4:
        df.loc[label, 'Classified'] = '可完整拆分'
    elif df.loc[label,'score'] == 0:
        df.loc[label, 'Classified'] = '無法拆分'
    else:
        df.loc[label, 'Classified'] = '可部分拆分'


In [16]:
df['Classified'].value_counts()

可完整拆分    1031
可部分拆分     653
無法拆分       28
Name: Classified, dtype: int64

In [17]:
df.to_excel('頁碼簿_分段狀況(測試).xlsx',index=False)
df.head(15)

,co_id,year,ext,filename,isedible,startpage,endpage,inputpath,filename_folder,text,exception,location,paragraph,score,sequence_projected,sequence_final,Classified
0,1102.0,2019,pdf,2019_1102_20200623F04_20211212_154547.pdf,True,5,7,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102/20...,2019_1102_20200623F04_20211212_154547.pdf,公司簡介 壹、致股東報告書 壹、2019 年國際暨國內經濟情勢回顧 ...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 167...",1.0,,,可部分拆分
1,1102.0,2020,pdf,2020_1102_20210625F04_20211212_154258.pdf,True,5,11,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102/20...,2020_1102_20210625F04_20211212_154258.pdf,致股東報告書 致股東報告書 壹、2020 年國際暨國內經濟情勢回顧 ...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 155...",3.0,,,可部分拆分
2,1103.0,2019,pdf,2019_1103_20200622F04_20211212_155216.pdf,True,4,5,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103/20...,2019_1103_20200622F04_20211212_155216.pdf,1 致股東報告書 致股東報告書 11. 致股東報告書各位股東女士、先生： (1) 主要產銷情...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 568...",2.0,,,可部分拆分
3,1103.0,2020,pdf,2020_1103_20210617F04_20211212_155303.pdf,True,6,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103/20...,2020_1103_20210617F04_20211212_155303.pdf,1.致股東報告書各位股東女士、先生： 民國(下同)109 年新冠病毒在全球大流行，各國...,~~dist too short,"{""1"": 1001, ""2"": 2656, ""3"": 2234, ""4"": 2162}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 100...",4.0,1432,1432,可完整拆分
4,1108.0,2019,pdf,2019_1108_20200618F04_20211212_155632.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1108/20...,2019_1108_20200618F04_20211212_155632.pdf,壹、致股東報告書 一、董事長致詞 各位股東女士、先生：大家好！ 本公司一○八年全年度合併營業...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 44,...",2.0,,,可部分拆分
5,1108.0,2020,pdf,2020_1108_20210616F04_20211212_155819.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1108/20...,2020_1108_20210616F04_20211212_155819.pdf,壹、致股東報告書 一、董事長致詞 各位股東女士、先生：大家好！ 本公司 109 年全年度合併...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 46,...",2.0,,,可部分拆分
6,1109.0,2019,pdf,2019_1109_20200623F04_20211212_155859.pdf,True,5,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1109/20...,2019_1109_20200623F04_20211212_155859.pdf,壹(cid:515)致股東報告書 各位股東女士(cid:501)先生： 108年國際...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 658...",3.0,,,可部分拆分
7,1109.0,2020,pdf,2020_1109_20210624F04_20211212_155958.pdf,True,5,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1109/20...,2020_1109_20210624F04_20211212_155958.pdf,壹(cid:515)致股東報告書 各位股東女士(cid:501)先生： 109 年國...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 525...",3.0,,,可部分拆分
8,1110.0,2019,pdf,2019_1110_20200620F04_20211212_160051.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1110/20...,2019_1110_20200620F04_20211212_160051.pdf,壹、致股東報告書 各位股東女士、先生： 依據台灣區水泥工業同業公會統計資料顯示：1...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 499...",2.0,,,可部分拆分
9,1110.0,2020,pdf,2020_1110_20210625F04_20211212_160217.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1110/20...,2020_1110_20210625F04_20211212_160217.pdf,壹、致股東報告書 各位股東女士、先生： 依據台灣區水泥工業同業公會統計資料顯示：1...,~~some situation not found,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 501...",2.0,,,可部分拆分


### 拆段落

In [18]:
# raise Exception

In [19]:
df1 = df.copy()
df1 = df1[df1['sequence_final']!='']
len(df1)

1031

In [20]:
def extract_paragraph(loc, fulltext, seq='1234'): 

    # 四個關鍵位置有任一個是-1，有段落合寫
    skip_seq = dict()
    for k,v in loc.items():
        if v==-1:
            skip_seq[k]=True
        else:
            skip_seq[k]=False
#     print(skip_seq)
    
    # 標註段落
    seq = seq[::-1] # reverse
    paragraph_text = {'1':'','2':'','3':'','4':''}
    post = len(fulltext)
    count = 0
    for s in seq:
#         print('skip?',skip_seq[s], end='\t')
#         print('post:',post, end='\t')
#         print('current:',loc[seq[count]], end='\t')
#         print('seq:',s, end='\n')
        if not skip_seq[s]:
            paragraph_text[s] = fulltext[loc[seq[count]]:post]
            post = loc[seq[count]] 
        count+=1
    paragraph_text['0'] = fulltext[:post]
    # 計算長度
    paragraph_length = {'0':0,'1':0,'2':0,'3':0,'4':0}
    for k,v in paragraph_text.items():
        paragraph_length[k] = len(v)
    
    return paragraph_text, paragraph_length

In [21]:
df1.head(5)

,co_id,year,ext,filename,isedible,startpage,endpage,inputpath,filename_folder,text,exception,location,paragraph,score,sequence_projected,sequence_final,Classified
3,1103.0,2020,pdf,2020_1103_20210617F04_20211212_155303.pdf,True,6,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103/20...,2020_1103_20210617F04_20211212_155303.pdf,1.致股東報告書各位股東女士、先生： 民國(下同)109 年新冠病毒在全球大流行，各國...,~~dist too short,"{""1"": 1001, ""2"": 2656, ""3"": 2234, ""4"": 2162}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 100...",4.0,1432,1432,可完整拆分
10,1203.0,2019,pdf,2019_1203_20200623F04_20211212_160843.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1200食品工業\1203/20...,2019_1203_20200623F04_20211212_160843.pdf,致股東報告書 壹、致股東報告書 一、前言 「民以食為天」，不論古今中外，三餐飲食都是每一個時...,,"{""1"": 207, ""2"": 1069, ""3"": 1427, ""4"": 2075}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 209...",4.0,1234,1234,可完整拆分
11,1203.0,2020,pdf,2020_1203_20210623F04_20211212_161121.pdf,True,5,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1200食品工業\1203/20...,2020_1203_20210623F04_20211212_161121.pdf,致股東報告書 壹、致股東報告書 一、前言 自從 2019 年底新冠肺炎疫情席捲全球，蔓延至今...,,"{""1"": 271, ""2"": 1165, ""3"": 1382, ""4"": 2203}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 273...",4.0,1234,1234,可完整拆分
12,1210.0,2019,pdf,2019_1210_20200605F04_20211212_161313.pdf,True,7,12,C:\Users\DC\辰瑜的論文\年報108-109年\M1200食品工業\1210/20...,2019_1210_20200605F04_20211212_161313.pdf,壹、致股東報告書各位股東女士、先生：大家好首先竭誠歡迎各位貴賓，在百忙當中特別是新冠病毒疫情...,,"{""1"": 353, ""2"": 876, ""3"": 2701, ""4"": 3266}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 353...",4.0,1234,1234,可完整拆分
13,1210.0,2020,pdf,2020_1210_20210611F04_20211212_161713.pdf,True,7,12,C:\Users\DC\辰瑜的論文\年報108-109年\M1200食品工業\1210/20...,2020_1210_20210611F04_20211212_161713.pdf,壹、致股東報告書各位股東女士、先生：大家好首先竭誠歡迎各位貴賓，在百忙當中特別是新冠病毒疫情...,,"{""1"": 369, ""2"": 910, ""3"": 3039, ""4"": 3694}","{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 369...",4.0,1234,1234,可完整拆分


In [22]:
# 開始分類每個檔案
count=0
pbar = tqdm(total=len(df1))
colist = []
for label, row in df1.iterrows():   
    try:
        location = json.loads(row['location'])
        text = row['text']
        sequence = row['sequence_final']
        pt, pl = extract_paragraph(location, text, sequence)
        
        colist.append({
            'file':row['inputpath'],
            'filename': row['filename_folder'],
            'text': row['text'],
            'location': row['location'],
            'exception':row['exception'],
            'lenpara0':pl['0'],
            'lenpara1':pl['1'],
            'lenpara2':pl['2'],
            'lenpara3':pl['3'],
            'lenpara4':pl['4'],
            'lentotal':len(row['text']),
            'para0':pt['0'],
            'para1':pt['1'],
            'para2':pt['2'],
            'para3':pt['3'],
            'para4':pt['4'],
            'sequence':sequence
        })
    except Exception as e:
        print(e)
        print(row['filename_folder'])
        
    pbar.update(1)
    
#     if count==15:
#         break
pbar.close()
dfr = pd.DataFrame(colist)

100%|███████████████████████████████████████████████████████████████████████████| 1031/1031 [00:00<00:00, 15195.26it/s]


In [23]:
dfr.to_excel('測試_分段結果.xlsx',index=False)

PermissionError: [Errno 13] Permission denied: '測試_分段結果.xlsx'

In [ ]:
dfr